# We want to scrape FBREF for defender statistics (defensive actions & miscellaneous stats that apply to defenders)

In [2]:
import pandas as pd

base_url = "https://fbref.com/en/comps/Big5/{season}/defense/players/{season}-Big-5-European-Leagues-Stats"

seasons = ["2023-2024", "2022-2023", "2021-2022"]

dfs = []

for season in seasons:
    url = base_url.format(season = season)
    
    try:
        tables = pd.read_html(url)
        df = tables[0]

        df.columns = df.columns.droplevel(0)
        df = df.rename(columns = lambda x: x if not x.startswith("Unnamed") else " ")

        df = df[~df["Pos"].str.contains("FW", na=False)]
        df = df[~df["Pos"].str.contains("GK", na=False)]

        df = df[df["Player"] != "Player"]  # Remove repeated header row
        df["Season"] = season  # Add a column to track the season

        df = df.reset_index(drop=True)  # Reset index
        
        dfs.append(df)  # Store the cleaned DataFrame
        
        print(f"Successfully fetched data for {season}")

    except Exception as e:
        print(f"Failed to fetch data for {season}: {e}")

final_df = pd.concat(dfs, ignore_index=True)

Failed to fetch data for 2023-2024: HTTP Error 429: Too Many Requests
Failed to fetch data for 2022-2023: HTTP Error 429: Too Many Requests
Failed to fetch data for 2021-2022: HTTP Error 429: Too Many Requests


ValueError: No objects to concatenate

In [4]:
import pandas as pd
base_url = "https://fbref.com/en/comps/Big5/{season}/defense/players/{season}-Big-5-European-Leagues-Stats"
all_player_dfs = []
seasons = ["2023-2024", "2022-2023", "2021-2022"]

for season in seasons:
    url = base_url.format(season = season)
    
    try:
        tables = pd.read_html(url)
        df = tables[0]

        df.columns = df.columns.droplevel(0)
        df = df.rename(columns = lambda x: x if not x.startswith("Unnamed") else " ")

        df = df[df["Player"] != "Player"]  # Remove repeated header row
        df["Season"] = season  # Add a column to track the season

        df = df.reset_index(drop=True)  # Reset index
        
        all_player_dfs.append(df)  # Store the cleaned DataFrame
        
        print(f"Successfully fetched data for {season}")

    except Exception as e:
        print(f"Failed to fetch data for {season}: {e}")

all_players = pd.concat(all_player_dfs, ignore_index=True)

Successfully fetched data for 2023-2024
Successfully fetched data for 2022-2023
Successfully fetched data for 2021-2022


# Now, we do the same for the FBREF page that contains the Miscellaneous stats.

In [2]:
base_misc_url = "https://fbref.com/en/comps/Big5/{season}/misc/players/{season}-Big-5-European-Leagues-Stats"

misc_seasons = ["2023-2024", "2022-2023", "2021-2022"]

misc_dfs = []

for season in misc_seasons:
    misc_url = base_misc_url.format(season = season)
    
    try:
        misc_tables = pd.read_html(misc_url)
        misc_df = misc_tables[0]

        misc_df.columns = misc_df.columns.droplevel(0)
        misc_df = misc_df.rename(columns = lambda x: x if not x.startswith("Unnamed") else " ")

        misc_df = misc_df[~misc_df["Pos"].str.contains("FW", na=False)]
        misc_df = misc_df[~misc_df["Pos"].str.contains("GK", na=False)]

        misc_df = misc_df[misc_df["Player"] != "Player"]  # Remove repeated header row
        misc_df["Season"] = season  # Add a column to track the season

        misc_df = misc_df.reset_index(drop=True)  # Reset index
        
        misc_dfs.append(misc_df)  # Store the cleaned DataFrame
        
        print(f"Successfully fetched data for {season}")

    except Exception as e:
        print(f"Failed to fetch data for {season}: {e}")

misc_final_df = pd.concat(misc_dfs, ignore_index=True)

Successfully fetched data for 2023-2024
Successfully fetched data for 2022-2023
Successfully fetched data for 2021-2022


In [1]:
base_misc_url = "https://fbref.com/en/comps/Big5/{season}/misc/players/{season}-Big-5-European-Leagues-Stats"
misc_seasons = ["2023-2024", "2022-2023", "2021-2022"]
all_misc_dfs = []

for season in misc_seasons:
    misc_url = base_misc_url.format(season = season)
    
    try:
        misc_tables = pd.read_html(misc_url)
        misc_df = misc_tables[0]

        misc_df.columns = misc_df.columns.droplevel(0)
        misc_df = misc_df.rename(columns = lambda x: x if not x.startswith("Unnamed") else " ")

        misc_df = misc_df[misc_df["Player"] != "Player"]  # Remove repeated header row
        misc_df["Season"] = season  # Add a column to track the season

        misc_df = misc_df.reset_index(drop=True)  # Reset index
        
        all_misc_dfs.append(misc_df)  # Store the cleaned DataFrame
        
        print(f"Successfully fetched data for {season}")

    except Exception as e:
        print(f"Failed to fetch data for {season}: {e}")

all_misc = pd.concat(all_misc_dfs, ignore_index=True)

Failed to fetch data for 2023-2024: name 'pd' is not defined
Failed to fetch data for 2022-2023: name 'pd' is not defined
Failed to fetch data for 2021-2022: name 'pd' is not defined


NameError: name 'pd' is not defined

# Now, we want to process the Defensive & Miscellaneous Stats to get rid of irrelevant & duplicate columns and rename columns for clarity:
- Player (Name)
- Nation
- Squad
- Comp
- Age
- Born
- PKs Won
- Fouls Drawn
- Matches

In [3]:
final_df.drop(columns=["Rk", "Nation", "Age", "Born", "Matches", "Season"], inplace=True)
final_df = final_df.rename(columns={})
misc_final_df.drop(columns=["Rk", "Pos", "90s", "Player", "Nation", "Squad", "TklW", "Int", "Comp", "Age", "Born", "PKwon", "Fld", "Matches"], inplace=True)

In [4]:
final_df_combined = pd.concat([final_df, misc_final_df], axis=1)
print(final_df_combined.shape)

(4937, 34)


# We also want to rename columns for clarity and to avoid duplicate column names

In [5]:
columns = list(final_df_combined.columns)
columns[10] = "Chl-Tkl"
columns[13] = "Chl-Lost"
final_df_combined.columns = columns

# We now convert all columns that contain numeric (string) values to numeric type

In [6]:
# Convert all columns in df_combined to numeric except the "pos" column
exclude_columns = ["Player", "Pos", "Season", "Comp", "Squad"]
final_df_combined.loc[:, ~final_df_combined.columns.isin(exclude_columns)] = (
    final_df_combined.loc[:, ~final_df_combined.columns.isin(exclude_columns)]
    .apply(pd.to_numeric, errors="coerce")
)

# Important considerations:
- Players with less than 5 total 90s played will be removed from the data
- Players who strictly qualify as "midfielders" might get removed in the future.

In [7]:
final_df_combined = final_df_combined.rename(columns={'Player': 'Player Name', 'Comp': 'League'})
filtered_df = final_df_combined[final_df_combined['90s'] > 5]
filtered_df.to_csv("polished_data_no_ratings.csv", index=False)

In [5]:
all_players

,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,90s,Tkl,...,Lost,Blocks,Sh,Pass,Int,Tkl+Int,Clr,Err,Matches,Season
0,1,Max Aarons,eng ENG,DF,Bournemouth,eng Premier League,23,2000,13.7,29,...,14,9,5,4,8,37,27,0,Matches,2023-2024
1,2,Brenden Aaronson,us USA,"MF,FW",Union Berlin,de Bundesliga,22,2000,14.1,32,...,16,26,1,25,2,34,4,0,Matches,2023-2024
2,3,Paxten Aaronson,us USA,MF,Eint Frankfurt,de Bundesliga,19,2003,1.1,2,...,0,2,0,2,0,2,0,0,Matches,2023-2024
3,4,Keyliane Abdallah,fr FRA,FW,Marseille,fr Ligue 1,17,2006,0.0,0,...,0,0,0,0,0,0,0,0,Matches,2023-2024
4,5,Yunis Abdelhamid,ma MAR,DF,Reims,fr Ligue 1,35,1987,30.9,64,...,19,51,32,19,39,103,109,2,Matches,2023-2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8657,2917,Martín Zubimendi,es ESP,MF,Real Sociedad,es La Liga,22,1999,28.8,52,...,22,30,13,17,49,101,48,1,Matches,2021-2022
8658,2918,Szymon Żurkowski,pl POL,MF,Empoli,it Serie A,23,1997,25.6,58,...,39,43,10,33,35,93,32,1,Matches,2021-2022
8659,2919,Martin Ødegaard,no NOR,"MF,FW",Arsenal,eng Premier League,22,1998,30.9,49,...,39,18,1,17,11,60,11,1,Matches,2021-2022
8660,2920,Milan Đurić,ba BIH,FW,Salernitana,it Serie A,31,1990,24.1,1,...,2,6,1,5,2,3,32,0,Matches,2021-2022


In [12]:
all_misc

,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,90s,CrdY,...,TklW,PKwon,PKcon,OG,Recov,Won,Lost,Won%,Matches,Season
0,1,Max Aarons,eng ENG,DF,Bournemouth,eng Premier League,23,2000,13.7,1,...,19,0,1,0,75,5,11,31.3,Matches,2023-2024
1,2,Brenden Aaronson,us USA,"MF,FW",Union Berlin,de Bundesliga,22,2000,14.1,3,...,18,0,0,0,88,13,16,44.8,Matches,2023-2024
2,3,Paxten Aaronson,us USA,MF,Eint Frankfurt,de Bundesliga,19,2003,1.1,0,...,2,0,0,0,5,3,0,100.0,Matches,2023-2024
3,4,Keyliane Abdallah,fr FRA,FW,Marseille,fr Ligue 1,17,2006,0.0,0,...,0,0,0,0,0,0,1,0.0,Matches,2023-2024
4,5,Yunis Abdelhamid,ma MAR,DF,Reims,fr Ligue 1,35,1987,30.9,5,...,35,0,0,1,149,61,37,62.2,Matches,2023-2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8657,2917,Martín Zubimendi,es ESP,MF,Real Sociedad,es La Liga,22,1999,28.8,4,...,25,0,0,0,139,63,24,72.4,Matches,2021-2022
8658,2918,Szymon Żurkowski,pl POL,MF,Empoli,it Serie A,23,1997,25.6,5,...,33,0,1,0,162,24,32,42.9,Matches,2021-2022
8659,2919,Martin Ødegaard,no NOR,"MF,FW",Arsenal,eng Premier League,22,1998,30.9,4,...,21,0,2,0,162,14,23,37.8,Matches,2021-2022
8660,2920,Milan Đurić,ba BIH,FW,Salernitana,it Serie A,31,1990,24.1,6,...,1,1,0,0,40,242,83,74.5,Matches,2021-2022


In [13]:
all_players.drop(columns=["Rk", "Nation", "Age", "Born", "Matches", "Season"], inplace=True)
all_players = all_players.rename(columns={})
all_misc.drop(columns=["Rk", "Pos", "90s", "Player", "Nation", "Squad", "TklW", "Int", "Comp", "Age", "Born", "PKwon", "Fld", "Matches"], inplace=True)

In [14]:
final_players = pd.concat([all_players, all_misc], axis=1)
print(final_players.shape)

(8662, 34)


In [16]:
cols = list(final_players.columns)
cols[10] = "Chl-Tkl"
cols[13] = "Chl-Lost"
final_players.columns = cols

In [17]:
# Convert all columns in df_combined to numeric except the "pos" column
exclude_columns = ["Player", "Pos", "Season", "Comp", "Squad"]
final_players.loc[:, ~final_players.columns.isin(exclude_columns)] = (
    final_players.loc[:, ~final_players.columns.isin(exclude_columns)]
    .apply(pd.to_numeric, errors="coerce")
)

In [18]:
final_players = final_players.rename(columns={'Player': 'Player Name', 'Comp': 'League'})
filtered_final = final_players[final_players['90s'] > 5]
filtered_final.to_csv("all_players_polished.csv", index=False)